<a href="https://colab.research.google.com/github/lingxiao254/comp5703_cs6_4/blob/master/RF_balanced_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Use GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import os
os.getcwd()
os.chdir('/content/drive/My Drive/')
os.getcwd()

'/content/drive/My Drive'

In [6]:
#import library
from glob import glob
import pandas as pd
import numpy as np
import fnmatch
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
x_sub=np.load('X.npy')
y_sub=np.load('Y.npy')

In [8]:
x_sub = np.array(x_sub)
x_sub_shape = x_sub.shape[1] * x_sub.shape[2] * x_sub.shape[3]
x_flat = x_sub.reshape(x_sub.shape[0], x_sub_shape)
r = pd.value_counts(y_sub)
print(r)

0    15718
1     4282
dtype: int64


In [9]:
rus=RandomUnderSampler(ratio='auto')
x_flat_resample,y_sub_resample=rus.fit_sample(x_flat,y_sub)
len(x_flat_resample)
r = pd.value_counts(y_sub_resample)
print(r)

1    4282
0    4282
dtype: int64


In [0]:
#split
x_train,x_test,y_train,y_test=train_test_split(x_flat_resample,y_sub_resample,test_size=0.2)

In [11]:
#random forest
rf=RandomForestClassifier(n_estimators=20,max_depth=50,max_features='auto')
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=50, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [12]:
test_prediction=rf.predict(x_test)
print(accuracy_score(y_test,test_prediction))
print(confusion_matrix(y_test,test_prediction))
print(metrics.classification_report(y_test,test_prediction))

0.8365440747227088
[[736 125]
 [155 697]]
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       861
           1       0.85      0.82      0.83       852

    accuracy                           0.84      1713
   macro avg       0.84      0.84      0.84      1713
weighted avg       0.84      0.84      0.84      1713



In [13]:
#Tuning Parameter
test1={'n_estimators':range(10,100,10)}
grid_search1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,min_samples_leaf=20,max_depth=8,max_features='auto' ,random_state=20),param_grid =test1, scoring='roc_auc',cv=5)
grid_search1.fit(x_train,y_train)
grid_search1.best_params_
rf1=RandomForestClassifier(n_estimators=90,max_depth=8,min_samples_split=100,max_features='auto')
rf1.fit(x_train,y_train)
test_prediction1=rf1.predict(x_test)
print('accuracy_1:',accuracy_score(y_test,test_prediction1))

accuracy_1: 0.8493870402802102


In [14]:
test2 = {'max_depth':range(50,201,50), 'min_samples_split':range(50,201,50)}
grid_search2= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 90,min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=20),param_grid=test2, scoring='roc_auc',iid=False, cv=5)
grid_search2.fit(x_train,y_train)
grid_search2.best_params_
rf2=RandomForestClassifier(n_estimators=90,max_depth=50,min_samples_split=50,max_features='auto')
rf2.fit(x_train,y_train)
test_prediction2=rf2.predict(x_test)
print('accuracy_2:',accuracy_score(y_test,test_prediction2))


accuracy_2: 0.8499708114419148


In [15]:
test3={'min_samples_split':range(10,100,30), 'min_samples_leaf':range(10,100,30)}
grid_search3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 90, max_depth=50,max_features='auto' ,oob_score=True, random_state=20),param_grid =test3, scoring='roc_auc',iid=False, cv=5)
grid_search3.fit(x_train,y_train)
grid_search3.best_params_

{'min_samples_leaf': 10, 'min_samples_split': 10}

In [16]:
rf3=RandomForestClassifier(n_estimators=90,max_depth=50,min_samples_split=10,min_samples_leaf=10,max_features='auto')
rf3.fit(x_train,y_train)
test_prediction3=rf3.predict(x_test)
print('accuracy_3:',accuracy_score(y_test,test_prediction3))
print(confusion_matrix(y_test,test_prediction3))
print(metrics.classification_report(y_test,test_prediction3))

accuracy_3: 0.8499708114419148
[[737 124]
 [133 719]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       861
           1       0.85      0.84      0.85       852

    accuracy                           0.85      1713
   macro avg       0.85      0.85      0.85      1713
weighted avg       0.85      0.85      0.85      1713

